# Introduction to AI Chat with Ollama! 🤖

Welcome to this fun tutorial where we'll learn how to chat with AI models using Python! We'll be using something called **Ollama**, which lets us run AI models right on our own computers.

In this tutorial, we'll learn:
1. How to start an Ollama server for local usage.
2. Managing LLM's and downloading open-source models.
3. Chat with AI about anything!
4. How to save our conversations with the AI

Let's get started! 🚀

### Import Libraries

In [2]:
import subprocess
import os
import socket
import time
import psutil

## 1. Starting the Ollama Server 🖥️

Before we can use any Ollama models, we need to make sure the Ollama server is running. Let's create a helper function that will:
1. Check if an Ollama server is already running
2. Start a new server only if needed
3. Keep track of the server process

This way, we won't accidentally start multiple servers!

In [3]:
def is_port_in_use(port: int = 11434) -> bool:
    """Check if the Ollama port is already in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

def is_ollama_running() -> bool:
    """Check if Ollama process is running"""
    for proc in psutil.process_iter(['name']):
        try:
            if proc.info['name'] == 'ollama':
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False

The `ensure_ollama_server()` function above does several important things:

1. **Checks if Server is Running**: It uses two methods to check if a server is already running:
   - Checks if the Ollama port (11434) is in use
   - Looks for an existing Ollama process

2. **Safe Server Start**: Only starts a new server if one isn't already running

3. **Error Handling**: 
   - Handles startup errors gracefully
   - Includes timeout protection
   - Cleans up failed starts

4. **Status Updates**: 
   - Provides clear feedback about server status
   - Shows the process ID when starting a new server

In [4]:
def ensure_ollama_server():
    """
    Ensures that an Ollama server is running.
    Returns the process object if a new server was started, None if server was already running.
    """
    # First check if the server is already running
    if is_port_in_use() or is_ollama_running():
        print("✅ Ollama server is already running!")
        return None
    
    try:
        # Start the Ollama server
        print("🚀 Starting Ollama server...")
        proc = subprocess.Popen(
            ["ollama", "serve"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            start_new_session=True
        )
        
        # Wait for the server to start (max 10 seconds)
        for _ in range(10):
            if is_port_in_use():
                print(f"✅ Ollama server started successfully (PID: {proc.pid})")
                return proc
            time.sleep(1)
            
        raise TimeoutError("Server didn't start within 10 seconds")
        
    except Exception as e:
        print(f"❌ Error starting Ollama server: {e}")
        if 'proc' in locals():
            proc.terminate()
        raise


In [5]:
# Start the server if it's not already running
ollama_process = ensure_ollama_server()

✅ Ollama server is already running!


##### You can run this cell multiple times safely - it won't create duplicate servers! 🚀

## 2. Managing Ollama Models 🚀

Before we start chatting with AI, let's learn how to manage our Ollama models. We'll cover:
1. Listing available models
2. Pulling new models
3. Removing models we don't need

Let's try these commands out!


In [6]:
# List all available Ollama models on your system 
!ollama ls

NAME                      ID              SIZE      MODIFIED   
llama3.2-vision:latest    6f2f9757ae97    7.8 GB    2 days ago    
llama3.1:latest           46e0c10c039e    4.9 GB    2 days ago    
llama3.2:1b               baf6a787fdff    1.3 GB    3 days ago    
llava:latest              8dd30f6b0cb1    4.7 GB    4 days ago    
qwen3:0.6b                7df6b6e09427    522 MB    4 days ago    


### Pulling New Models 📥

You can pull various models from Ollama's model library https://ollama.com/library . Here are some popular ones:
- `phi`: Microsoft's Phi-2 model
- `llama2`: Meta's LLaMA 2 model
- `mistral`: Mistral AI's model
- `qwen`: Alibaba's Qwen model
- `neural-chat`: Intel's Neural Chat model

Let's try pulling a model!

In [9]:
# Pull the llama3.2:1b model (this might take a few minutes depending on your internet connection)
!ollama pull llama3.2:1b


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 1.1 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   0% ▕                  ▏ 2.1 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   1% ▕                  ▏  14 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   3% ▕                  ▏  45 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   6% ▕█                 ▏  77 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 158 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  19% ▕███               ▏ 254 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  23% ▕████              ▏ 309 MB/1.3 GB                  pulling manifest 
pulling 74701a8c

### Removing Models 🗑️

If you want to free up space, you can remove models you're not using. Here's how to do it:

In [ ]:
# Remove a model (uncomment and replace MODEL_NAME with the model you want to remove)
# !ollama rm MODEL_NAME

# For example, to remove the phi model:
# !ollama rm phi

## 3. Simple Chat with AI 💬

Let's start by importing the packages we need and creating a simple function to chat with our AI. We'll make it super easy to use!

In [8]:
import os
from datetime import datetime
from ollama import chat

def chat_with_ai(prompt: str, model: str = "qwen3:0.6b", temperature: float = 0.7):
    """
    Have a simple chat with an AI model!
    
    Args:
        prompt: Your question or message to the AI
        model: Which AI model to use (default is qwen3:0.6b)
        temperature: How creative the AI should be (0.0 = very focused, 1.0 = very creative)
    
    Returns:
        The AI's response
    """
    try:
        # Send our message to the AI
        response = chat(
            model=model,
            options={"temperature": temperature},
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Get the AI's response
        return response["message"]["content"]
    
    except Exception as e:
        return f"Oops! Something went wrong: {e}"

# Let's test our function with a simple question!
response = chat_with_ai("What is astrophysics? Explain it to a high school student. Respond in 3 sentences")
print("AI's response:\n", response)

AI's response:
 <think>
Okay, the user wants me to explain astrophysics to a high school student in three sentences. Let me start by breaking down what astrophysics is. First, it's the study of the universe, right? So I should mention the universe and the elements like stars, planets, etc. Then, I need to explain the main areas. Maybe start with the basics: stars and how they form. Then, the universe beyond stars, like galaxies and dark matter. Finally, mention that it's a huge field with lots of study. Let me check if each sentence connects and if it's clear. Yeah, that should work. Make sure the explanation is simple and uses everyday language.
</think>

Astrophysics is the study of the universe, including everything from stars and planets to galaxies and dark matter. It helps us understand how stars form, the processes that shape the universe, and the role of dark matter and energy. By studying these, scientists learn how the cosmos grew and how life might develop in the future.


## 4. Saving Our Conversations 📝

It's often useful to save our conversations with the AI for later. Let's create a function that helps us do that!


In [9]:
def save_chat(model: str, prompt: str, response: str):
    """
    Save our chat to a file so we can look at it later!
    
    Args:
        model: The AI model we used
        prompt: What we asked the AI
        response: What the AI answered
    """
    # Create a folder to store our chats if it doesn't exist
    results_dir = os.path.join("results", "chats")
    os.makedirs(results_dir, exist_ok=True)
    
    # Create a filename with the current time
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(results_dir, f"chat_{timestamp}.txt")
    
    # Save the conversation
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"Model: {model}\n")
        f.write(f"\nOur Question:\n{prompt}\n")
        f.write(f"\nAI's Answer:\n{response}\n")
    
    print(f"\nChat saved to: {filename}")

# Let's try asking another question and save it!
prompt = "What are three fun projects a high school student could do to learn about AI? Respond in 3 sentences."
response = chat_with_ai(prompt)
print("AI's response:\n", response)
save_chat("qwen3:0.6b", prompt, response)


AI's response:
 <think>
Okay, the user is asking for three fun projects a high school student could do to learn about AI. Let me think about how to structure this.

First, I need to make sure each project is engaging and fun for high school students. Maybe start with something interactive. Like building a simple AI model or creating an AI-based app. That's a common and fun idea. Then, another project could involve coding a basic AI algorithm. Since high school students might not be familiar with coding, using visual tools like Python or Scratch could be helpful. Finally, a third project could be making a personal AI assistant. That's a practical and fun application.

Wait, are there any other ideas? Maybe a project on data analysis with AI. But the user specified three, so I need to keep them distinct. Let me check each sentence to ensure clarity and that each project is a unique, fun endeavor. Also, make sure the sentences flow well and each project is clearly explained.
</think>

1. 